# RBE Estimator

> Main recursive Bayesian estimator implementations with performance metrics

In [ ]:
#| default_exp rbe.estimator

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from typing import Optional, Callable, Dict, List, Tuple
from fastcore.test import test_eq, test_close
from fastcore.all import *
from technical_blog.rbe.particle_filter import run as pf_run, init as pf_init, step as pf_step
from technical_blog.rbe.probability import eff_size

## Standard RBE Estimator

In [ ]:
#| export
def estimate(observations, transition_fn, likelihood_fn, 
            n_particles=1000, init_fn=None, rng=None):
    "Main RBE estimator for `observations` with particle filter"
    return pf_run(observations, transition_fn, likelihood_fn,
                  n_particles, init_fn, rng=rng)

In [ ]:
# Test basic estimator
rng = np.random.default_rng(42)

# Simple tracking problem
def test_transition(particle, rng):
    return particle + rng.normal(0, 0.05, particle.shape)

def test_likelihood(particle, observation):
    diff = np.linalg.norm(particle - observation)
    return np.exp(-0.5 * (diff / 0.1)**2)

# Generate test data
true_states = [np.array([i * 0.1, i * 0.05]) for i in range(10)]
observations = [state + rng.normal(0, 0.1, 2) for state in true_states]

# Run estimator
result = estimate(observations, test_transition, test_likelihood, 
                  n_particles=100, rng=rng)

assert result['estimates'].shape == (10, 2)
assert 'particles' in result
assert 'weights' in result
assert 'eff_sizes' in result

## Adaptive RBE Estimator

In [ ]:
#| export
def adaptive(observations, transition_fn, likelihood_fn, 
            adapt_fn=None, n_particles=1000, init_fn=None, rng=None):
    "Adaptive RBE estimator that adjusts parameters based on performance"
    if rng is None: rng = np.random.default_rng()
    if adapt_fn is None:
        # Default adaptation: adjust resampling threshold based on ESS
        def adapt_fn(ess_history, current_threshold):
            avg_ess = np.mean(ess_history[-5:]) if len(ess_history) >= 5 else ess_history[-1]
            if avg_ess < 0.3 * n_particles:
                return min(0.9, current_threshold + 0.1)
            elif avg_ess > 0.7 * n_particles:
                return max(0.1, current_threshold - 0.1)
            return current_threshold
    
    # Initialize
    state_dim = len(observations[0]) if hasattr(observations[0], '__len__') else 1
    particles, weights = pf_init(n_particles, state_dim, init_fn, rng)
    
    # Adaptive parameters
    resample_threshold = 0.5
    
    # Storage
    estimates = []
    ess_history = [eff_size(weights)]
    threshold_history = [resample_threshold]
    
    for obs in observations:
        # Adapt threshold
        resample_threshold = adapt_fn(ess_history, resample_threshold)
        threshold_history.append(resample_threshold)
        
        # Standard step with adaptive threshold
        particles, weights = pf_step(particles, weights, obs,
                                    transition_fn, likelihood_fn,
                                    resample_threshold, rng)
        
        # Record
        estimate = np.average(particles, weights=weights, axis=0)
        estimates.append(estimate)
        ess_history.append(eff_size(weights))
    
    return {
        'estimates': np.array(estimates),
        'ess_history': np.array(ess_history),
        'threshold_history': np.array(threshold_history),
        'final_particles': particles,
        'final_weights': weights
    }

In [ ]:
# Test adaptive estimator
adaptive_result = adaptive(observations, test_transition, test_likelihood,
                          n_particles=100, rng=rng)

assert adaptive_result['estimates'].shape == (10, 2)
assert len(adaptive_result['ess_history']) == 11  # Initial + 10 steps
assert len(adaptive_result['threshold_history']) == 11
print(f"Threshold adapted from {adaptive_result['threshold_history'][0]:.2f} to {adaptive_result['threshold_history'][-1]:.2f}")

## Performance Metrics

In [ ]:
#| export
def metrics(true_states, estimates):
    "Calculate performance metrics for RBE estimates"
    true_states = np.array(true_states)
    estimates = np.array(estimates)
    
    # Ensure same shape
    if true_states.shape != estimates.shape:
        raise ValueError(f"Shape mismatch: true {true_states.shape} vs estimates {estimates.shape}")
    
    errors = true_states - estimates
    
    # Mean squared error
    mse = np.mean(errors**2)
    
    # Root mean squared error
    rmse = np.sqrt(mse)
    
    # Mean absolute error
    mae = np.mean(np.abs(errors))
    
    # Maximum absolute error
    max_ae = np.max(np.abs(errors))
    
    # Per-dimension metrics if multidimensional
    if errors.ndim > 1:
        dim_mse = np.mean(errors**2, axis=0)
        dim_rmse = np.sqrt(dim_mse)
        dim_mae = np.mean(np.abs(errors), axis=0)
    else:
        dim_mse = mse
        dim_rmse = rmse
        dim_mae = mae
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'max_absolute_error': max_ae,
        'n_samples': len(estimates),
        'dim_mse': dim_mse,
        'dim_rmse': dim_rmse,
        'dim_mae': dim_mae
    }

In [ ]:
# Test metrics
perf_metrics = metrics(true_states, result['estimates'])
assert 'mse' in perf_metrics
assert 'rmse' in perf_metrics
assert 'mae' in perf_metrics
assert perf_metrics['n_samples'] == 10
assert perf_metrics['rmse'] == np.sqrt(perf_metrics['mse'])

print(f"RMSE: {perf_metrics['rmse']:.4f}")
print(f"MAE: {perf_metrics['mae']:.4f}")
print(f"Per-dimension RMSE: {perf_metrics['dim_rmse']}")

## Online vs Batch Comparison

In [ ]:
#| export
def compare_online_batch(observations, transition_fn, likelihood_fn,
                        n_particles=1000, rng=None):
    "Compare online (recursive) vs batch processing performance"
    if rng is None: rng = np.random.default_rng()
    
    # Online/recursive processing
    online_result = estimate(observations, transition_fn, likelihood_fn,
                            n_particles, rng=rng)
    
    # Batch processing (process all observations at once)
    # For particle filter, this means running forward-backward smoothing
    # Here we'll approximate with a simple batch mean for comparison
    state_dim = len(observations[0]) if hasattr(observations[0], '__len__') else 1
    particles, weights = pf_init(n_particles * 10, state_dim, rng=rng)  # More particles for batch
    
    # Apply all observations at once (simplified batch processing)
    for obs in observations:
        # Update weights based on all observations
        for i, particle in enumerate(particles):
            weights[i] *= likelihood_fn(particle, obs)
    
    weights = weights / np.sum(weights) if np.sum(weights) > 0 else np.ones_like(weights) / len(weights)
    batch_estimate = np.average(particles, weights=weights, axis=0)
    
    # For fair comparison, repeat batch estimate for all time steps
    batch_estimates = np.tile(batch_estimate, (len(observations), 1))
    
    return {
        'online': online_result,
        'batch': {
            'estimates': batch_estimates,
            'final_estimate': batch_estimate
        }
    }

In [ ]:
# Compare online vs batch
comparison = compare_online_batch(observations, test_transition, test_likelihood,
                                 n_particles=50, rng=rng)

online_metrics = metrics(true_states, comparison['online']['estimates'])
batch_metrics = metrics(true_states, comparison['batch']['estimates'])

print(f"Online RMSE: {online_metrics['rmse']:.4f}")
print(f"Batch RMSE: {batch_metrics['rmse']:.4f}")
print(f"Online better by: {(batch_metrics['rmse'] - online_metrics['rmse']) / batch_metrics['rmse'] * 100:.1f}%")

## Multi-Model RBE

In [ ]:
#| export
def multi_model(observations, models, model_probs=None, n_particles=1000, rng=None):
    "RBE with multiple competing models"
    if rng is None: rng = np.random.default_rng()
    n_models = len(models)
    
    if model_probs is None:
        model_probs = np.ones(n_models) / n_models
    
    # Run each model
    model_results = []
    model_likelihoods = []
    
    for model in models:
        result = estimate(observations, model['transition'], model['likelihood'],
                         n_particles // n_models, rng=rng)
        model_results.append(result)
        
        # Compute model likelihood (simplified - product of weights)
        likelihood = 1.0
        for weights in result['weights'][1:]:  # Skip initial
            likelihood *= np.mean(weights)  # Simplified
        model_likelihoods.append(likelihood)
    
    # Update model probabilities
    model_likelihoods = np.array(model_likelihoods)
    model_probs = model_probs * model_likelihoods
    model_probs = model_probs / np.sum(model_probs) if np.sum(model_probs) > 0 else np.ones(n_models) / n_models
    
    # Combine estimates using model probabilities
    combined_estimates = np.zeros_like(model_results[0]['estimates'])
    for i, (result, prob) in enumerate(zip(model_results, model_probs)):
        combined_estimates += prob * result['estimates']
    
    return {
        'estimates': combined_estimates,
        'model_probs': model_probs,
        'model_results': model_results
    }

In [ ]:
# Test multi-model estimator
models = [
    {
        'name': 'Low noise',
        'transition': lambda x, rng: x + rng.normal(0, 0.01, x.shape),
        'likelihood': test_likelihood
    },
    {
        'name': 'High noise', 
        'transition': lambda x, rng: x + rng.normal(0, 0.1, x.shape),
        'likelihood': test_likelihood
    }
]

multi_result = multi_model(observations[:5], models, n_particles=100, rng=rng)
assert multi_result['estimates'].shape == (5, 2)
assert len(multi_result['model_probs']) == 2
test_close(np.sum(multi_result['model_probs']), 1.0)
print(f"Model probabilities: {multi_result['model_probs']}")

## Export

In [ ]:
#| export
__all__ = [
    # Estimators
    'estimate', 'adaptive', 'multi_model',
    
    # Analysis
    'metrics', 'compare_online_batch'
]